In [1]:
import sys
import os
import pandas as pd
# 현재 작업 디렉토리 기준으로 src 디렉토리 추가
notebook_dir = os.getcwd()  # 현재 Jupyter Notebook의 작업 디렉토리

sys.path.append(os.path.dirname(notebook_dir))


In [2]:
from src.tools.google_drive import ensure_data_files

# 데이터 파일 경로 가져오기
data_paths = ensure_data_files()
data_paths


기존 data가 존재합니다. 파일 경로를 반환합니다.


{'reviewer': '/Users/seongrok.kim/Github/yamyam-lab/data/reviewer.csv',
 'review_keyword': '/Users/seongrok.kim/Github/yamyam-lab/data/review_keyword.csv',
 'category': '/Users/seongrok.kim/Github/yamyam-lab/data/diner_category_raw.csv',
 'review': '/Users/seongrok.kim/Github/yamyam-lab/data/review.csv',
 'dinr_menu': '/Users/seongrok.kim/Github/yamyam-lab/data/dinr_menu.csv',
 'diner': '/Users/seongrok.kim/Github/yamyam-lab/data/diner.csv'}

In [ ]:
# load data
diner = pd.read_csv(data_paths["diner"])
diner_category = pd.read_csv(data_paths["category"])

diner = pd.merge(diner, diner_category, on="diner_idx", how="left")

/var/folders/dq/xrxyf7x57q504wf515d672p00000gn/T/ipykernel_63861/1634552960.py:2: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner = pd.read_csv(data_paths["diner"])


In [4]:
from src.preprocess.diner_transform import CategoryProcessor

processor = CategoryProcessor(diner_category)

diner_with_processd_category = processor.process_all().df
diner = pd.merge(diner, diner_with_processd_category, on="diner_idx", how="left")


In [5]:
diner_with_processd_category.columns

Index(['diner_idx', 'industry_category', 'diner_category_large',
       'diner_category_middle', 'diner_category_small',
       'diner_category_detail'],
      dtype='object')

In [26]:
def extract_category_paths_with_counts(df):
    category_cols = [
        'industry_category',
        'diner_category_large',
        'diner_category_middle',
        'diner_category_small',
        'diner_category_detail',
    ]

    all_levels = []

    for i in range(1, len(category_cols) + 1):
        subset_cols = category_cols[:i]
        # Null 포함된 행 제거
        subset_df = df[subset_cols + ['diner_idx']].dropna()
        # 각 조합별로 음식점 수 카운트
        count_df = (
            subset_df
            .groupby(subset_cols, dropna=False)
            .agg(diner_count=('diner_idx', 'nunique'))  # 고유 음식점 수
            .reset_index()
        )
        count_df["depth"] = i
        all_levels.append(count_df)

    category_with_counts = pd.concat(all_levels, ignore_index=True)
    return category_with_counts


In [27]:
category_stats = extract_category_paths_with_counts(diner_with_processd_category)

# 상위 20개, 음식점 수 기준 정렬 예시
category_stats.sort_values(by='diner_count', ascending=False).head(20)


,industry_category,diner_count,depth,diner_category_large,diner_category_middle,diner_category_small,diner_category_detail
2,음식점,232587,1,NaN,NaN,NaN,NaN
13,음식점,107138,2,한식,NaN,NaN,NaN
119,음식점,34713,3,한식,"육류,고기",NaN,NaN
9,음식점,30445,2,양식,NaN,NaN,NaN
7,음식점,28703,2,술집,NaN,NaN,NaN
5,음식점,19746,2,간식,NaN,NaN,NaN
127,음식점,17445,3,한식,"해물,생선",NaN,NaN
12,음식점,16327,2,카페,NaN,NaN,NaN
108,음식점,14493,3,한식,분식,NaN,NaN
39,음식점,13108,3,술집,"호프,요리주점",NaN,NaN


In [28]:
category_stats

,industry_category,diner_count,depth,diner_category_large,diner_category_middle,diner_category_small,diner_category_detail
0,"가정,생활",452,1,NaN,NaN,NaN,NaN
1,"서비스,산업",5,1,NaN,NaN,NaN,NaN
2,음식점,232587,1,NaN,NaN,NaN,NaN
3,"가정,생활",452,2,기타,NaN,NaN,NaN
4,"서비스,산업",5,2,기타,NaN,NaN,NaN
...,...,...,...,...,...,...,...
955,음식점,35,5,한식,"해물,생선",추어,남가네설악추어탕
956,음식점,17,5,한식,"해물,생선",추어,송담추어탕
957,음식점,3,5,한식,"해물,생선",추어,원조설악추어탕
958,음식점,21,5,한식,"해물,생선",추어,춘향골남원추어탕


In [30]:
category_stats[category_stats['diner_category_small'] == '추어']

,industry_category,diner_count,depth,diner_category_large,diner_category_middle,diner_category_small,diner_category_detail
711,음식점,1031,4,한식,"해물,생선",추어,NaN
955,음식점,35,5,한식,"해물,생선",추어,남가네설악추어탕
956,음식점,17,5,한식,"해물,생선",추어,송담추어탕
957,음식점,3,5,한식,"해물,생선",추어,원조설악추어탕
958,음식점,21,5,한식,"해물,생선",추어,춘향골남원추어탕


In [31]:
category_stats.to_csv('category_table.csv', index=False, encoding='utf-8-sig')